# Challenge 1 - Tic Tac Toe

In this lab you will perform deep learning analysis on a dataset of playing [Tic Tac Toe](https://en.wikipedia.org/wiki/Tic-tac-toe).

There are 9 grids in Tic Tac Toe that are coded as the following picture shows:

![Tic Tac Toe Grids](tttboard.jpg)

In the first 9 columns of the dataset you can find which marks (`x` or `o`) exist in the grids. If there is no mark in a certain grid, it is labeled as `b`. The last column is `class` which tells you whether Player X (who always moves first in Tic Tac Toe) wins in this configuration. Note that when `class` has the value `False`, it means either Player O wins the game or it ends up as a draw.

Follow the steps suggested below to conduct a neural network analysis using Tensorflow and Keras. You will build a deep learning model to predict whether Player X wins the game or not.

## Step 1: Data Engineering

This dataset is almost in the ready-to-use state so you do not need to worry about missing values and so on. Still, some simple data engineering is needed.

1. Read `tic-tac-toe.csv` into a dataframe.
1. Inspect the dataset. Determine if the dataset is reliable by eyeballing the data.
1. Convert the categorical values to numeric in all columns.
1. Separate the inputs and output.
1. Normalize the input data.

In [27]:
# your code here
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from tensorflow.keras.models import Sequential # Normal neural network
from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Dropout, Input
from keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical


In [2]:
data = pd.read_csv("tic-tac-toe.csv")

display(data.sample(10))
display(data.info())
# A quick look shows that whenever the class is True, there is indeed three X together. When the class is False, there are three O together

,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
460,o,b,o,x,x,x,o,x,b,True
167,x,o,o,x,b,b,x,x,o,True
520,b,x,b,o,x,o,b,x,b,True
351,o,x,b,o,x,x,b,x,o,True
253,x,b,o,x,o,b,x,b,b,True
495,b,x,x,b,o,x,o,o,x,True
878,o,b,b,b,o,x,x,x,o,False
764,o,x,x,o,b,o,o,x,x,False
417,o,o,b,o,x,b,x,x,x,True
184,x,o,o,b,x,b,b,b,x,True


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   TL      958 non-null    object
 1   TM      958 non-null    object
 2   TR      958 non-null    object
 3   ML      958 non-null    object
 4   MM      958 non-null    object
 5   MR      958 non-null    object
 6   BL      958 non-null    object
 7   BM      958 non-null    object
 8   BR      958 non-null    object
 9   class   958 non-null    bool  
dtypes: bool(1), object(9)
memory usage: 68.4+ KB


None

In [3]:
data.replace({'b':0, 'x':1, 'o':-1}, inplace=True)
display(data.sample(10))
display(data.info())

C:\Users\Saul\AppData\Local\Temp\ipykernel_23844\3281563554.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data.replace({'b':0, 'x':1, 'o':-1}, inplace=True)


,TL,TM,TR,ML,MM,MR,BL,BM,BR,class
668,1,1,0,-1,-1,-1,0,1,0,False
213,1,-1,0,-1,1,0,0,0,1,True
615,0,0,-1,-1,1,-1,1,1,1,True
581,0,-1,0,1,1,1,0,0,-1,True
676,1,-1,1,1,-1,0,-1,-1,1,False
924,0,-1,0,1,-1,1,1,-1,0,False
854,-1,0,1,1,-1,0,1,0,-1,False
836,-1,-1,-1,0,1,1,1,-1,1,False
405,-1,-1,1,0,0,1,-1,1,1,True
523,0,1,0,0,1,-1,-1,1,0,True


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 958 entries, 0 to 957
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   TL      958 non-null    int64
 1   TM      958 non-null    int64
 2   TR      958 non-null    int64
 3   ML      958 non-null    int64
 4   MM      958 non-null    int64
 5   MR      958 non-null    int64
 6   BL      958 non-null    int64
 7   BM      958 non-null    int64
 8   BR      958 non-null    int64
 9   class   958 non-null    bool 
dtypes: bool(1), int64(9)
memory usage: 68.4 KB


None

In [4]:
inputs = data.iloc[:,:-1]
outputs = data["class"]
display(inputs, outputs)

,TL,TM,TR,ML,MM,MR,BL,BM,BR
0,1,1,1,1,-1,-1,1,-1,-1
1,1,1,1,1,-1,-1,-1,1,-1
2,1,1,1,1,-1,-1,-1,-1,1
3,1,1,1,1,-1,-1,-1,0,0
4,1,1,1,1,-1,-1,0,-1,0
...,...,...,...,...,...,...,...,...,...
953,-1,1,1,1,-1,-1,-1,1,1
954,-1,1,-1,1,1,-1,1,-1,1
955,-1,1,-1,1,-1,1,1,-1,1
956,-1,1,-1,-1,1,1,1,-1,1


0       True
1       True
2       True
3       True
4       True
       ...  
953    False
954    False
955    False
956    False
957    False
Name: class, Length: 958, dtype: bool

In [14]:
# Because of the values I chose to replace x and o, this step is actually not needed
scaler = StandardScaler()
standard_inputs = scaler.fit_transform(inputs)

## Step 2: Build Neural Network

To build the neural network, you can refer to your own codes you wrote while following the [Deep Learning with Python, TensorFlow, and Keras tutorial](https://www.youtube.com/watch?v=wQ8BIBpya2k) in the lesson. It's pretty similar to what you will be doing in this lab.

1. Split the training and test data.
1. Create a `Sequential` model.
1. Add several layers to your model. Make sure you use ReLU as the activation function for the middle layers. Use Softmax for the output layer because each output has a single lable and all the label probabilities add up to 1.
1. Compile the model using `adam` as the optimizer and `sparse_categorical_crossentropy` as the loss function. For metrics, use `accuracy` for now.
1. Fit the training data.
1. Evaluate your neural network model with the test data.
1. Save your model as `tic-tac-toe.model`.

In [15]:
# 1. Split the training and test data
X_train, X_test, y_train, y_test = train_test_split(standard_inputs,outputs)

In [34]:
# 2. Create a `Sequential` model
model = Sequential()

In [35]:
# 3. Add several layers to your model
batch_size = 64 # The neural network will adapt the weights every batch_size number of data points (instead of running all the data points and then changing the weights)
num_classes = 2 # Number of classes (similar to labels in regression)
epochs = 50 # Number of forward and backward propagations

model.add(Input(shape=(9,))) # This is preferable to input_shape in the first layer in order to avoid a UserWarning
model.add(Dense(16, activation='relu')) # Adds a layer of 16 neurons, with a ReLU activation function.
model.add(Dropout(0.2)) # Randomly chooses 20% of the weights after a layer and sets them to 0 to avoid overfitting
model.add(Dense(32, activation='relu')) # Adds a layer of 32 neurons, with a ReLU activation function. It doesn't need an input shape because it will just take the inputs from the preceeding layer
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax')) # Adds a layer of num_classes neurons, with a softmax activation function, which returns the biggeset of [P(class=0), P(class=1), P(class=-1)].

model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_6 (Dense)                 │ (None, 16)             │           160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 32)             │           544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 2)              │            66 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 770 (3.01 KB)

 Trainable params: 770 (3.01 KB)

 Non-trainable params: 0 (0.00 B)

In [38]:
# 4. Compile the model
optimizer = Adam(learning_rate=0.002)

model.compile(loss='sparse_categorical_crossentropy',
              optimizer = optimizer,  
              metrics=['accuracy'])

In [39]:
# 5. Fit the training data
model.fit(X_train, y_train,
         batch_size=batch_size,
         epochs=epochs,
         verbose=1, # How much information the algorithm will print. If 0, it won't print anything; if 1, it will print info about every epoch
         validation_data=(X_test, y_test))

Epoch 1/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 4s 52ms/step - accuracy: 0.8760 - loss: 0.3111 - val_accuracy: 0.9708 - val_loss: 0.1792
Epoch 2/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.9208 - loss: 0.2532 - val_accuracy: 0.9792 - val_loss: 0.1557
Epoch 3/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9087 - loss: 0.2446 - val_accuracy: 0.9833 - val_loss: 0.1363
Epoch 4/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9196 - loss: 0.2265 - val_accuracy: 0.9917 - val_loss: 0.1184
Epoch 5/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 30ms/step - accuracy: 0.9166 - loss: 0.2276 - val_accuracy: 0.9958 - val_loss: 0.1027
Epoch 6/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step - accuracy: 0.9180 - loss: 0.2222 - val_accuracy: 0.9958 - val_loss: 0.0890
Epoch 7/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.9164 - loss: 0.2076 - val_accuracy: 0.9958 - val_loss: 0.0823
Epoch 8/50
12/12 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - accuracy: 0.9305 - loss: 0.2029 - val_accuracy: 0.9958 - v

In [40]:
# 6. Evaluate your neural network model
score = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', score[0]) # Similar to error
print('Test accuracy:', score[1])

Test loss: 0.018685080111026764
Test accuracy: 0.9958333373069763


In [22]:
# 7. Save your model
model.save("tic-tac-toe.keras")

## Step 3: Make Predictions

Now load your saved model and use it to make predictions on a few random rows in the test dataset. Check if the predictions are correct.

In [33]:
# your code here
model = load_model("tic-tac-toe.keras")

predictions = model.predict(X_test) # Returns an array of sub-arrays. Each sub-array contains the probability of each label for every data point
display(np.argmax(predictions, axis=1)) # Returns the index of the biggest probability
display(np.array(y_test))

8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step


array([1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1,
       0, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1],
      dtype=int64)

array([ True,  True,  True,  True,  True,  True, False,  True,  True,
        True,  True,  True,  True, False,  True,  True,  True, False,
        True,  True, False,  True,  True,  True, False, False,  True,
        True,  True, False,  True,  True,  True,  True,  True, False,
       False,  True,  True, False,  True,  True,  True,  True,  True,
        True,  True, False,  True, False, False,  True, False,  True,
       False, False,  True, False,  True, False, False,  True,  True,
       False, False, False,  True,  True,  True,  True,  True, False,
        True,  True,  True, False,  True,  True, False,  True,  True,
       False, False,  True, False, False,  True,  True, False,  True,
       False, False,  True,  True,  True,  True, False,  True, False,
        True,  True,  True, False, False,  True, False,  True,  True,
       False, False, False,  True,  True,  True,  True,  True,  True,
       False,  True, False, False,  True,  True,  True, False, False,
       False, False,

## Step 4: Improve Your Model

Did your model achieve low loss (<0.1) and high accuracy (>0.95)? If not, try to improve your model.

But how? There are so many things you can play with in Tensorflow and in the next challenge you'll learn about these things. But in this challenge, let's just do a few things to see if they will help.

* Add more layers to your model. If the data are complex you need more layers. But don't use more layers than you need. If adding more layers does not improve the model performance you don't need additional layers.
* Adjust the learning rate when you compile the model. This means you will create a custom `tf.keras.optimizers.Adam` instance where you specify the learning rate you want. Then pass the instance to `model.compile` as the optimizer.
    * `tf.keras.optimizers.Adam` [reference](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers/Adam).
    * Don't worry if you don't understand what the learning rate does. You'll learn about it in the next challenge.
* Adjust the number of epochs when you fit the training data to the model. Your model performance continues to improve as you train more epochs. But eventually it will reach the ceiling and the performance will stay the same.

In [ ]:
# your code here
# Instead of rewriting the code, I applied these changes in the code from the previous challenges

**Which approach(es) did you find helpful to improve your model performance?**

In [ ]:
# your answer here
"""
Increasing the number of epochs
Increasing the learning rate of the optimizer
"""